Name: Temirbekova Dilnaz

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
#print(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
data = pd.read_csv('./data.tsv',sep='\t')
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
print("Unique Values of Sentiment are: {}".format(", ".join(list(map(str,data["Sentiment"].unique())))))

Unique Values of Sentiment are: 1, 2, 3, 4, 0


In [5]:
x = data["Phrase"].tolist()
y = data["Sentiment"].apply(lambda i: 0 if i <= 2 else 1)

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
lemmatizer = WordNetLemmatizer()
def proc_text(messy):
    first = BeautifulSoup(messy, "lxml").get_text()
    second = re.sub("[^a-zA-Z]"," ",first) 
    third = second.lower().split() 
    fourth = set([lemmatizer.lemmatize(str(x)) for x in third])
    stops = set(stopwords.words("english"))
    almost = [w for w in fourth if not w in stops]
    final = " ".join(almost)
    return final
x = [proc_text(i) for i in x]

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=100, test_size=0.2, stratify=y)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

124848
31212
124848
31212


In [22]:
cat_data=data.groupby('SentenceId')['Phrase'].sum().to_frame()
cat_data.head()

,Phrase
SentenceId,
1,A series of escapades demonstrating the adage ...
2,"This quiet , introspective and entertaining in..."
3,"Even fans of Ismail Merchant 's work , I suspe..."
4,A positively thrilling combination of ethnogra...
5,Aggressive self-glorification and a manipulati...


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X1 = vectorizer.fit_transform(cat_data['Phrase'])
#print(vectorizer.get_feature_names())
print(X1.toarray())
print(X1.shape)
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(cat_data['Phrase'])
#print(vectorizer2.get_feature_names())
print(X2.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(8529, 92118)
(8529, 261987)


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
pos_weights = (len(y_train) - sum(y_train)) / (sum(y_train)) 
pipeline_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', DecisionTreeClassifier(random_state=100, class_weight={0: 1, 1: pos_weights}))
])
pipeline_tf.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier',
                 DecisionTreeClassifier(class_weight={0: 1,
                                                      1: 2.7040289562689135},
                                        random_state=100))])

In [17]:
predictions = pipeline_tf.predict(x_test)
predicted_proba = pipeline_tf.predict_proba(x_test)

In [21]:
from sklearn.metrics import accuracy_score
print("Accuracy score is about {}%".format(accuracy_score(y_test, predictions)*100))

Accuracy score is about 82.74702037677815%
